In [4]:
import tensorflow as tf
import pandas as pd
import numpy as np

### GPU 설정

In [2]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0' :
    raise SystemError('GPU device not found')
print(f'Found GPU at: {device_name}')

Found GPU at: /device:GPU:0


In [3]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6604417257618315380
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 2255906407
locality {
  bus_id: 1
  links {
  }
}
incarnation: 4452680535721767156
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1650 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5"
xla_global_id: 416903419
]


### SGD를 사용한 MF 기본 알고리즘

In [5]:
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('./data/u.data', names=r_cols, sep='\t', encoding='latin-1')
ratings = ratings[['user_id', 'movie_id', 'rating']].astype(int)

In [ ]:
# MF class
class MF() : 
    def __init__(self, ratings, k, alpha, beta, iterations, verbose=True) : 
        self.R = np.array(ratings)
        self.num_users, self.num_items = np.shape(self.R)
        # 아래는 파라미터로 받아본 변수를 클래스에 저장하는 부분
        self.K = k
        self.alpha = alpha
        self.beta = beta 
        self.iterations = iterations
        self.verbose = verbose
    
    # RMSE 계산
    def rmse(self) : 
        xs, ys = self.R.nonzero()
        self.predictions = []
        self.errors = []
        for x, y in zip(xs, ys) : 
            prediction = self.get_prediction(x, y)
            self.predictions.append(prediction)
            self.errors.append(self.R[x,y] - prediction)
        self.predictions = np.array(self.predictions)
        self.errors = np.array(self.errors)
        return np.sqrt(np.mean(self.errors**2))
     
    def train(self) :
        # 사용자-특징, 영화-특징 행렬 초기화
        self.P = np.random.normal(scale=1) 
    


